In [12]:
import pandas as pd
import time
import torch
from torch.utils.data import TensorDataset, DataLoader

In [2]:
df = pd.read_csv('/kaggle/input/simps/data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [3]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [4]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [5]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}
INDEX_TO_CHARS = {w: i for i, w in CHAR_TO_INDEX.items()}

In [6]:
def caesar(text, shift): 
    caesarText = ""
    for ch in text:
        if ch.isalpha():
            caesar_ord = ord(ch) + shift 
            if caesar_ord > ord('z'):
                caesar_ord -= 26
            caesar_ch = chr(caesar_ord)
            caesarText += caesar_ch
        else:
            caesarText += ch  

    return caesarText

In [7]:
caesar('abc xyz', 3)

'def abc'

In [8]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
y = torch.zeros((len(text), MAX_LEN), dtype=int)
X = torch.zeros((len(text), MAX_LEN), dtype=int)# создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    for j, w in enumerate(caesar(text[i],3)):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])    

In [9]:
y[0], X[0]

(tensor([21, 26,  5,  5,  2, 10, 13, 16,  1,  1, 24, 13, 11, 12, 26, 17,  4, 13,
         17, 12, 26, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 tensor([20,  6, 27, 27, 16, 12, 13,  1,  7,  7, 14, 13, 19, 24,  6, 11, 15, 13,
         11, 24,  6, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]))

In [10]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
batch_size = 256
data_train = TensorDataset(X_train, y_train)
data_test = TensorDataset(X_test, y_test)
iter_train = DataLoader(data_train, batch_size, shuffle=True)
iter_test = DataLoader(data_test, batch_size)

In [57]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(len(INDEX_TO_CHAR), 50)
        self.rnn = torch.nn.RNN(50, 128, batch_first=True)
        self.out = torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x)
        return self.out(x)

In [24]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(INDEX_TO_CHAR), 256)
        self.rnn = torch.nn.RNN(256, 512, batch_first=True)
        self.linear = torch.nn.Linear(512, len(INDEX_TO_CHAR))
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [58]:
model = Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [59]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    test_loss = 0.
    test_passed = 0
    
    model.train()
    for X_batch,y_batch in iter_train:
        optimizer.zero_grad()
        y_batch = y_batch.view(1, -1).squeeze()
        y_pred = model(X_batch)
        y_pred = y_pred.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(y_pred, y_batch)  
        train_loss += loss.item()
        train_passed += 1
        loss.backward()
        optimizer.step()    
        
    model.eval()
    for X, y in iter_test:
        optimizer.zero_grad()
        y_batch = y_batch.view(1, -1).squeeze()
        y_pred = model(X_batch)
        y_pred = y_pred.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(y_pred, y_batch)  
        test_loss += loss.item()
        test_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}, Test loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed, test_loss / test_passed))

Epoch 0. Time: 1.703, Train loss: 1.552, Test loss: 0.523
Epoch 1. Time: 1.797, Train loss: 0.371, Test loss: 0.180
Epoch 2. Time: 1.725, Train loss: 0.095, Test loss: 0.038
Epoch 3. Time: 1.721, Train loss: 0.044, Test loss: 0.072
Epoch 4. Time: 1.848, Train loss: 0.026, Test loss: 0.024
Epoch 5. Time: 2.023, Train loss: 0.018, Test loss: 0.014
Epoch 6. Time: 1.713, Train loss: 0.013, Test loss: 0.013
Epoch 7. Time: 1.769, Train loss: 0.010, Test loss: 0.019
Epoch 8. Time: 1.707, Train loss: 0.008, Test loss: 0.006
Epoch 9. Time: 1.716, Train loss: 0.007, Test loss: 0.018


## Проверим работу модели на случайном предложении

In [110]:
test_sentence = 'caesar cipher one of simplest encryption technique'
len(test_sentence),caesar(test_sentence, 3)

(50, 'fdhvdu flskhu rqh ri vlpsohvw hqfubswlrq whfkqltxh')

In [111]:
test = torch.tensor([CHAR_TO_INDEX[i] for i in test_sentence])
test

tensor([25, 26, 10,  4, 26,  7, 13, 25,  2, 20, 12, 10,  7, 13,  1, 14, 10, 13,
         1,  8, 13,  4,  2, 21, 20, 16, 10,  4, 17, 13, 10, 14, 25,  7,  3, 20,
        17,  2,  1, 14, 13, 17, 10, 25, 12, 14,  2, 22, 23, 10])

In [112]:
test_caesar = torch.tensor([CHAR_TO_INDEX[i] for i in caesar(test_sentence, 3)])
test_caesar

tensor([ 8,  6, 12, 15,  6, 23, 13,  8, 16,  4, 24, 12, 23, 13,  7, 22, 12, 13,
         7,  2, 13, 15, 16, 20,  4,  1, 12, 15, 11, 13, 12, 22,  8, 23, 18,  4,
        11, 16,  7, 22, 13, 11, 12,  8, 24, 22, 16, 17,  9, 12])

In [113]:
test_pred = model(test_caesar)
pred = test_pred.argmax(dim=1)
pred

tensor([25, 26, 10,  4, 26,  7, 13, 25,  2, 20, 12, 10,  7, 13,  1, 14, 10, 13,
         1,  8, 13,  4,  2, 21, 20, 16, 10,  4, 17, 13, 10, 14, 25,  7,  3, 20,
        17,  2,  1, 14, 13, 17, 10, 25, 12, 14,  2, 22, 23, 10])

In [114]:
pred = [int(i) for i in pred]
pred = [INDEX_TO_CHARS[i] for i in pred]
pred_sentence = ''.join(pred)
pred_sentence

'caesar cipher one of simplest encryption technique'

In [115]:
test_sentence == pred_sentence

True